# Лаб 1. Корреляционная система

При создании задания использовались Visual Studio Code и Python 3.8.

## Теория

Данные для рекомендательных систем представляют из себя матрицу $R$ юзер-айтемов, в строках которой находятся пользователи, а в столбцах объекты. В ячейках матрицы находятся оценки пользователей (или какая-то другая метрика $-$ просмотры, покупки и т.д.):

$$
R = \begin{pmatrix}
   r_{1 1} & r_{1 2} & ...     &    \\
   r_{2 1} & r_{2 1} & ...     &     \\
   ...     & ...     & r_{u i} & ... \\
           &         & ...     &     \\
\end{pmatrix}
$$

Матрица $R$ сильно разрежена, то есть большая часть ячеек пуста, поскольку каждый пользователь взаимодействовал с очень малым числом объектов. Задача $-$ предсказать значения в пустых ячейках, то есть получить новую заполненную матрицу $\hat R$, максимально похожую на $R$.

### Обозначения

- $U - $ множество пользователей;
- $I - $ множество айтемов (объектов);
- $I(u)$, где $u \in U -$ множество айтемов, которое оценил пользователь $u$. То есть такие айтемы, для которых в строке $u$ матрицы $R$ не пусто. Аналогично $I(u, v) -$ множество объектов, которые оценили и $u$ и $v$;
- $U(u)$, где $u \in U -$ Множество пользователей, оценивали то же что и $u$, то есть $I(u) \cap I(v) \ne \empty$, где $v \in U(u)$;
- $r_u$, где $u \in U -$ строка матрицы $R$, соответствующая пользователю $u$;
- $r_i$, где $i \in I -$ столбец матрицы $R$, соответствующий айтему $i$;
- $\bar r_u$, где $u \in U -$ среднее значение по всем заполненным оценкам пользователя $u$.


### Корреляционная модель

Один из самых простых способов $-$ использовать взвешенное среднее. То есть, чтобы определить оценку пользователя $u$ айтема $i$, надо усреднить оценки всех пользователей, посмотревших этот фильм (user-based подход). При этом будем учитывать оценку пользователя с большим коэффициентом если пользователь "похож" на нашего:

$$
\hat{r}_{ui} = \frac{
        \sum_{v \in U(u)} S(u, v)\cdot r_v
    } {
        \sum_{v \in U(u)} S(u, v)
    }
$$

Здесь $S$ это функция близости, которая тем больше, чем более "похожи" пользователи друг на друга.

Поскольку оценки разных пользователей могут отличаться $-$ кто-то ставит всем фильмам 9 и 10, а кто-то 0 и 1, можно попытаться устранить проблему, предсказывая не само значение $r_u$, а отклонение от среднего значения $(r_u - \bar{r}_u)$:

$$
\hat{r}_{ui} = \bar{r}_{u} + \frac{
        \sum_{v \in U(u)} S(u, v) \cdot (r_v - \bar r_v)
    } {
        \sum_{v \in U(u)} S(u, v)
    }
$$



### Функция сходства

Будем рассматривать функцию сходства двух юзером для user-based модели, функции сходства для айтемов определяется аналогично.

#### Косинусная мера сходства

Считаем косинус угла между пользователями в пространстве определяющих их векторов. То есть берём скалярное произведение и делим на длины векторов. Единственное, на что следует обратит внимание, что берутся не все оценки пользователей, а только для тех айтемов, которые они оба оценили:

$$
S(u, v) = \frac{
        \sum_{i \in I(u, v)} r_{ui}r_{vi}
    }{
        \sqrt{\sum_{i \in I(u)} r_{ui}^2}
        \sqrt{\sum_{i \in I(v)} r_{vi}^2}
    }
$$


## Задание

Дан датасет с оценками пользователями фильмов. Реализуйте алгоритм в соответствии с вашим вариантом и порекомендуйте себе фильмы. Сделайте выводы.

В датасете фильмы оценены по пятибальной шкале. Если в вашем варианте используется функция сходства для бинарных данных, используйте факт просмотра фильма (наличие оценки вообще).

### Варианты

1. User-based подход, Косинусная мера сходства.
2. User-based подход, Корреляция Пирсона.
3. User-based подход, Мера близости Жаккара.
4. Item-based подход, Косинусная мера сходства.
5. Item-based подход, Корреляция Пирсона
6. Item-based подход, Мера близости Жаккара.


## Код

Устанавливаем библиотеки

In [1]:
#%pip install pandas tqdm

Импорты

In [2]:
# Пандас нам нужен для загрузки csv и работы с матрицей
import pandas as pd
# Это библиотека для визуализации прогресса в питоновском ноутбуке
from tqdm.notebook import tqdm

import math
import time
import numpy as np

Загружаем датасет

In [3]:
# Сам датасет
ratings = pd.read_csv('./ml-latest-small/ratings.csv', delimiter=',')
# Оставляем в таблице только нужные столбцы
ratings = ratings.loc[:, ["userId", "movieId", "rating"]]

# Строка в таблице в датасета это id пользователя, id фильма, рейтинг
# На id пользователей нам плевать, а фильмы хочется смотреть по названиям,
# поэтому загружаем табличку сопоставления названий фильмов их id
movies = pd.read_csv('./ml-latest-small/movies.csv', delimiter=',')

In [4]:
# Делаем таблицу для преобразования имени в id
title_to_id = movies.loc[:, ["title", "movieId"]]
title_to_id.set_index("title", inplace=True)

# Делаем таблицу для преобразования id в имя
id_to_title = movies.loc[:, ["movieId", "title"]]
id_to_title.set_index("movieId", inplace=True)

# Проверяем что всё работает
print(title_to_id.loc[["Iron Man (2008)", "Doctor Who: A Christmas Carol (2010)"], :])
print(id_to_title.loc[[59315, 147376], :])


                                      movieId
title                                        
Iron Man (2008)                         59315
Doctor Who: A Christmas Carol (2010)   147376
                                        title
movieId                                      
59315                         Iron Man (2008)
147376   Doctor Who: A Christmas Carol (2010)


In [5]:
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [16]:
# Мы хотим получить для себя какую-то рекомендацию,
# для этого оценим несколько фильмов по пятибальной шкале
# (аккуратно копипастим имена из датасета)
my_ratings = {
        "Toy Story (1995)": 3.0,
        "Jumanji (1995)": 2.0,
        "It Takes Two (1995)": 1.0,
        "Mortal Kombat (1995)": 2.0,
        "Taxi Driver (1976)": 5.0,
        "Scarface (1983)": 5.0,
        "Saw (2004)": 4.0,
      "xXx: State of the Union (2005)": 0.0,
      "V for Vendetta (2006)": 4.0,
      "Night at the Museum (2006)": 3.0,
      "Pirates of the Caribbean: At World's End (2007)": 5.0,
    "Harry Potter and the Order of the Phoenix (2007)":2.0,
    "Resident Evil: Extinction (2007)":3.0,
    "Zebraman (2004)":2.0,
    "Max Payne (2008)":3.0,
    "Madagascar: Escape 2 Africa (2008)":4.0,
    "Shrek the Halls (2007)":2.0,
    "Drive (2011)":5.0,
    "Casino (1995)":5.0,
    "Star Wars: Episode IV - A New Hope (1977)": 2.0,
    "Star Wars: Episode I - The Phantom Menace (1999)": 3.0,
    "LEGO Batman: The Movie - DC Heroes Unite (2013)": 4.0,
    "Lord of the Rings: The Fellowship of the Ring, The (2001)": 4.0,
    "Back to the Future (1985)": 4.0,
    "Iron Man (2008)": 5.0,
    "Avengers, The (2012)": 2.0, 
}

# Даём нашему юзеру id, которого нет в датасете
my_user_id = 666
# Докидываем свои оценки в датасет
for m, r in my_ratings.items():
    mid = title_to_id.loc[m]["movieId"]
    row = pd.DataFrame([[my_user_id, mid, r]], columns=["userId", "movieId", "rating"])
    ratings = pd.concat([ratings, row])
ratings=ratings.reset_index(drop=True)

In [17]:
ratings.loc[ratings['userId'] == 666]['movieId']

100836    147376
100837       260
100838      2628
100839    103233
100840      4993
100841      1270
100842     59315
100843     89745
100844    179819
100845         1
100846         2
100847        38
100848        44
100849       111
100850      4262
100851      8957
100852     33158
100853     44191
100854     46972
100855     53125
100856     54001
100857     55232
100858     58870
100859     62394
100860     62999
100861     64249
100862     88129
100863        16
100864       260
100865      2628
100866    103233
100867      4993
100868      1270
100869     59315
100870     89745
Name: movieId, dtype: int64

In [18]:
ratings.loc[(ratings['userId'] == 666)&(ratings['movieId']==260)]['rating'].reset_index(drop=True)[0]

5.0

In [19]:
ratings.loc[ratings['userId'] == 666]["movieId"].isin([260]).any()

True


### Функция сходства

Будем рассматривать функцию сходства двух юзером для user-based модели, функции сходства для айтемов определяется аналогично.

#### Косинусная мера сходства

Считаем косинус угла между пользователями в пространстве определяющих их векторов. То есть берём скалярное произведение и делим на длины векторов. Единственное, на что следует обратит внимание, что берутся не все оценки пользователей, а только для тех айтемов, которые они оба оценили:

$$
S(u, v) = \frac{
        \sum_{i \in I(u, v)} r_{ui}r_{vi}
    }{
        \sqrt{\sum_{i \in I(u)} r_{ui}^2}
        \sqrt{\sum_{i \in I(v)} r_{vi}^2}
    }
$$



### Корреляционная модель

Один из самых простых способов $-$ использовать взвешенное среднее. То есть, чтобы определить оценку пользователя $u$ айтема $i$, надо усреднить оценки всех пользователей, посмотревших этот фильм (user-based подход). При этом будем учитывать оценку пользователя с большим коэффициентом если пользователь "похож" на нашего:

$$
\hat{r}_{ui} = \frac{
        \sum_{v \in U(u)} S(u, v)\cdot r_v
    } {
        \sum_{v \in U(u)} S(u, v)
    }
$$

Здесь $S$ это функция близости, которая тем больше, чем более "похожи" пользователи друг на друга.

Поскольку оценки разных пользователей могут отличаться $-$ кто-то ставит всем фильмам 9 и 10, а кто-то 0 и 1, можно попытаться устранить проблему, предсказывая не само значение $r_u$, а отклонение от среднего значения $(r_u - \bar{r}_u)$:

$$
\hat{r}_{ui} = \bar{r}_{u} + \frac{
        \sum_{v \in U(u)} S(u, v) \cdot (r_v - \bar r_v)
    } {
        \sum_{v \in U(u)} S(u, v)
    }
$$


In [20]:
# Тут реализуем функцию предсказания.
# Или класс. Главное - предсказать.
def S (u,v):
    s_up=0
    s_down1=0
    s_down2=0
    a= list(set( ratings.loc[ratings['userId'] == u]['movieId']).intersection(ratings.loc[ratings['userId'] == v]['movieId']))
    for i in a:
        s_up+=(ratings.loc[(ratings['userId'] == v)&(ratings['movieId']==i)]['rating'].reset_index(drop=True)[0]*
               ratings.loc[(ratings['userId'] == u)&(ratings['movieId']==i)]['rating'].reset_index(drop=True)[0])
        s_down1+= ratings.loc[(ratings['userId'] == u)&(ratings['movieId']==i)]['rating'].reset_index(drop=True)[0]**2
        s_down2+= ratings.loc[(ratings['userId'] == v)&(ratings['movieId']==i)]['rating'].reset_index(drop=True)[0]**2
        #print (s_up/((s_down1*s_down1)**(0.5)))
    return (s_up/((s_down1*s_down2)**(0.5)))
def r_mid(u):
    return ratings.loc[ratings['userId'] == u]["rating"].mean()
def predict(user_id, movie_id):
    pred=0
    if (ratings.loc[ratings['userId'] == user_id]["movieId"].isin([movie_id]).any()):
        pred=ratings.loc[(ratings['userId'] == user_id)&(ratings['movieId']==movie_id)]['rating'].reset_index(drop=True)[0]
    else:
        r_up=0
        r_down=0
        for i in (ratings.loc[ratings['movieId'] == movie_id]['userId']):
            a= list(set( ratings.loc[ratings['userId'] == user_id]['movieId']).intersection(ratings.loc[ratings['userId'] == i]['movieId']))
            if a:
                s=S(user_id,i)
                r_up += s*(ratings.loc[(ratings['userId'] == i)&(ratings['movieId']==movie_id)]['rating'].reset_index(drop=True)[0]-r_mid(i))
                r_down+=abs(s)
        if r_down==0:
            pred=0.0
        else:
            pred=r_mid(user_id)+r_up/r_down
    #print (r_up)
    return pred


Даём предсказание для каждого фильма

In [21]:
result = {}
for m in tqdm(ratings['movieId'].drop_duplicates().reset_index(drop=True)):
    result[m] = predict(my_user_id, m)
    #print(result.items())
result = {k: v for k, v in result.items()}

  0%|          | 0/9724 [00:00<?, ?it/s]

KeyboardInterrupt: 

Выводим результат

In [ ]:
# Преобразуем id фильмов в нормальные названия
human_readable_result = {}
for m, v in result.items():
    title = id_to_title.loc[m]["title"]
    human_readable_result[title] = v

# Сортируем массив с результатами по убыванию
sorted_result = {k: v for k, v in sorted(human_readable_result.items(), key=lambda item: item[1], reverse=True) if v != 0.0}

# Выводим первые 20 рекомендаций
for m, v in list(sorted_result.items())[:20]:
    print(m, ": ", v)


## Выводы

Объясните полученный результат: